# Tent (帐篷)

规则和案例可以见：

思维游戏“帐篷”的规则很简单:

- 在与树水平或竖直方向相邻的格子里放置帐篷。帐篷与树一一对应。
- 帐篷之间互不相邻（即便是对角也不行）
- 边框外的数字表示每行、每列的帐篷总数。(最后的实例忘记截了...)

-----

输入数据：

1. 图的规模；
2. 树的位置；
3. 行、列外的数字

输出数据：

帐篷的位置

-------------



The rules of the logic game "Tents" are very simple:

- Place tents in squares adjacent horizontally or vertically to trees. Each tent corresponds to a tree.
- Tents cannot be adjacent to each other (not even diagonally).
- Numbers outside the grid indicate the total number of tents in each row and column. (The last example was forgotten to be cut off...)

-----

Input Data:

1. Size of the grid.
2. Positions of the trees.
3. Numbers outside the rows and columns.

Output Data:

Positions of the tents.


<https://cn.puzzle-tents.com/?size=8>



In [50]:

def readGrid(path):
    with open(f"../assets/data/tent/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        rows = f.readline().strip().split(" ")
        cols = f.readline().strip().split(" ")
        grid = f.readlines()
        res = [list(map(int, g.strip().split(" "))) for g in grid]
        return int(m), int(n), [int(r) for r in rows], [int(c) for c in cols], res

if __name__ == "__main__":
    m, n, rows, cols, res = readGrid("30x30_1")
    print(m)
    print(n)
    print(rows)
    print(cols)
    print(len(res))

30
30
[12, 1, 9, 4, 5, 8, 4, 6, 5, 7, 5, 5, 7, 4, 7, 6, 6, 4, 8, 2, 8, 5, 4, 10, 2, 8, 5, 8, 3, 12]
[11, 2, 6, 7, 4, 6, 7, 6, 6, 7, 3, 8, 4, 7, 4, 8, 4, 7, 6, 4, 6, 6, 6, 6, 7, 4, 7, 6, 3, 12]
30


In [51]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp

class TentSolver:
    
    def __init__(self, X, Y, rows, cols, grid) -> None:
        self.X = X
        self.Y = Y 
        self.rows = rows
        self.cols = cols 
        self.grid = grid
        self.x = {}
        self.avail_x = {}
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        
        
    def getNeighbours(self, pos):
        
        neighbours = []
        directions = [[1,0], [-1,0], [0,1], [0,-1]]
        for direction in directions:
            if pos[0] + direction[0] >= self.X or pos[0] + direction[0] < 0 \
                or pos[1] + direction[1] >= self.Y or pos[1] + direction[1] < 0 \
                    or self.grid[pos[0] + direction[0]][pos[1] + direction[1]] == 1:
                    continue
            neighbours.append((pos[0] + direction[0], pos[1] + direction[1]))
        
        return neighbours

    def solve(self):

        for i in range(self.X):
            for j in range(self.Y):
                self.avail_x[i, j] = []
        for i in range(self.X):
            for j in range(self.Y):
                
                if self.grid[i][j] == 1:
                    neighbours = self.getNeighbours((i, j))
                    avail_cages = []
                    for neighbour in neighbours:
                        self.x[neighbour[0], neighbour[1], i * self.Y + j] = self.model.NewBoolVar(name = f"x[{neighbour[0]}, {neighbour[1]}, {i * self.Y + j}]")
                        avail_cages.append(self.x[neighbour[0], neighbour[1], i * self.Y + j])
                        self.avail_x[neighbour[0], neighbour[1]].append(self.x[neighbour[0], neighbour[1], i * self.Y + j])
                    self.model.Add(sum(avail_cages) == 1)
        
        for idx, row in enumerate(rows):
            curr_lst = []
            for i in range(self.X):
                if self.grid[i][idx] == 0: 
                    curr_lst += self.avail_x[i, idx]
            self.model.Add(sum(curr_lst) == row)
        
        for i in range(self.X):
            for j in range(self.Y):
                if self.grid[i][j] == 0:
                    self.model.Add(sum(self.avail_x[i, j]) <= 1)
                    
        for idx, col in enumerate(cols):
            curr_lst = []
            for j in range(self.Y):
                if self.grid[idx][j] == 0: 
                    curr_lst += self.avail_x[idx, j]
            self.model.Add(sum(curr_lst) == col)
        
        
        for i in range(self.X - 1):
            for j in range(self.Y - 1):
                self.model.Add(sum(self.avail_x[i, j] + self.avail_x[i, j + 1] + self.avail_x[i + 1, j] + self.avail_x[i + 1, j + 1]) <= 1)
            
        status = self.solver.Solve(self.model)
        if status == cp.OPTIMAL:
            for i in range(self.X):
                for j in range(self.Y):
                    if self.grid[i][j] == 1:
                        print("T",  end=" ")
                    else:
                        isFill = False
                        for k in self.avail_x[i, j]:
                            if self.solver.Value(k) == 1:
                                print(self.solver.Value(k), end = " ")
                                isFill = True
                                break
                        if not isFill:
                            print("0", end = " ")
                print()
            print()
        print(self.solver.ResponseStats())
            

if __name__ == "__main__":
    m, n, rows, cols, res = readGrid("30x30_1")
    TentSolverTest = TentSolver(m, n, rows, cols, res)
    TentSolverTest.solve()
    
# TODO: Modify Input struction like:
# """
# 00000011000100
# .....  
# """
# Like wise. 

1 0 0 T 1 0 T 1 0 1 0 T 1 0 0 1 T 1 T 1 T 1 0 1 0 0 T 0 1 T 
T T 1 0 0 0 0 0 0 T 0 0 T 0 0 T 0 0 0 0 0 0 0 T 0 0 1 0 0 T 
0 0 0 T 1 0 0 0 1 T 1 0 1 0 0 T 0 0 0 0 1 0 0 0 0 0 0 0 0 1 
1 T 1 T 0 0 1 T 0 0 T 0 T 0 0 1 0 1 T 0 T 0 0 1 T 1 0 0 0 T 
0 T 0 0 0 0 0 0 T 1 0 0 1 0 T 0 0 0 0 0 0 0 0 T 0 0 0 1 T 1 
0 1 0 1 T 0 0 0 0 0 0 0 0 0 1 0 1 T 0 T 1 0 0 1 T T 0 0 0 0 
T 0 0 0 0 T 1 T 1 0 1 0 0 0 0 0 0 0 1 0 0 0 T 0 0 1 0 1 T 1 
1 0 T 0 1 0 0 0 0 0 T 0 0 1 0 1 T 0 T 0 1 0 1 0 0 0 0 0 0 T 
T 0 1 0 T T 0 T T 1 0 0 0 T T 0 0 0 1 0 T 0 0 0 0 1 T 1 T 1 
1 0 0 0 0 1 0 1 0 0 0 0 1 T 1 0 0 0 T T 0 1 T 1 0 T 0 0 0 0 
0 0 0 0 0 0 0 T 0 0 0 0 0 0 0 0 0 1 T 1 0 T 0 0 0 0 0 1 0 0 
0 0 1 T 0 0 0 1 0 0 1 0 0 1 T 1 T T 0 0 0 T 1 0 T 1 0 T 0 1 
1 T 0 0 0 1 0 0 0 0 T 0 0 0 0 0 0 1 0 0 T 0 0 0 0 0 0 1 T T 
0 0 0 1 T T 0 T 0 1 T 1 0 0 0 0 T 0 0 0 1 0 T 1 0 1 T 0 T 1 
T 0 0 0 0 1 0 1 0 T 0 0 0 0 1 T 1 0 0 0 0 0 0 T 0 0 0 0 0 0 
1 0 0 0 0 T 0 0 0 1 T T 1 0 0 0 0 T 1 0 T 1 0 1 0 0 1 T 1 0 
0 0 1 T 1 0 1 T 0 0 0 T 

1 T 1 T 0 T 1 T 1 0 
T 0 0 0 1 0 0 T 0 0 
1 1 0 T T T 0 1 T 1 
0 T 0 1 0 1 0 0 0 0 
0 0 0 T 1 0 0 0 0 1 
0 0 0 0 0 0 0 1 T T 
T 1 0 0 0 0 0 0 0 0 
0 0 0 1 T 0 0 0 0 0 
T 0 0 T T 1 0 1 T 0 
1 0 0 1 0 0 0 0 0 0 



![](../assets/figures/Tent%201%20New.png)
